In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [5]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.257093                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.103687
0.000750      2.0  0.256263                  2.0  0.106803
0.001052      3.0  0.265227                  3.0  0.115285
0.001476      4.0  0.273815                  4.0  0.124222
0.002070      5.0  0.280512                  5.0  0.132927
0.002904      6.0  0.284934                  6.0  0.141156
0.004074      7.0  0.286165                  7.0  0.148630
0.005714      8.0  0.283415                  8.0  0.154841
0.008015      9.0  0.275624                  9.0  0.159241
0.011243     10.0  0.261654                 10.0  0.160835
0.015771     11.0  0.240291                 11.0  0.158469
0.022122     12.0  0.210238                 12.0  0.150258
0.031031     13.0  0.169938                 13.0  0.133935
0.043528     14.0  0.117514                 14.0  0.106924
0.061057     15.0  0.077121                 15.0  0.088715
0.085645     16.0  0.092819                 16.0  0.117851
0.120136     17.0  0.147272                 17.0  0.180819
0.168516     18.0  0.191082                 18.0  0.225568
0.236378     19.0  0.226419                 19.0  0.241674
0.331549     20.0  0.264553                 20.0  0.244762
0.465100     21.0  0.321568                 21.0  0.274006
0.652400     22.0  0.431150                 22.0  0.383781
0.915100     23.0  0.569781                 23.0  0.540986
1.283650     24.0  0.553692                 24.0  0.545302
1.800600     25.0  0.421327                 25.0  0.427306
2.525700     26.0  0.318857                 26.0  0.327479
3.542800     27.0  0.244444                 27.0  0.258080
4.969550     28.0  0.188558                 28.0  0.207038
6.970850     29.0  0.147734                 29.0  0.167702
9.778100     30.0  0.116244                 30.0  0.130238
13.715850    31.0  0.088009                 31.0  0.093332
19.239350    32.0  0.062248                 32.0  0.059131
26.987250    33.0  0.044936                 33.0  0.036594
37.855300    34.0  0.035011                 34.0  0.026977
53.100050    35.0  0.024979                 35.0  0.021471
73.887500    36.0  0.014264                 36.0  0.014482
97.662500    37.0  0.007843                 37.0  0.008897
121.437500   38.0  0.004521                 38.0  0.004716
145.212500   39.0 -0.002173                 39.0 -0.003327
168.987500   40.0 -0.010065                 40.0 -0.012464
192.762500   41.0 -0.009398                 41.0 -0.012628
216.537500   42.0  0.000392                 42.0 -0.003490
240.312500   43.0  0.009710                 43.0  0.005579
264.087500   44.0  0.018398                 44.0  0.013925
287.862500   45.0  0.027410                 45.0  0.022274
311.637500   46.0  0.036360                 46.0  0.030491
335.412500   47.0  0.045296                 47.0  0.038721
359.187500   48.0  0.054391                 48.0  0.047306
382.962500   49.0  0.063594                 49.0  0.056297
406.737500   50.0  0.072775                 50.0  0.065503
430.512500   51.0  0.081748                 51.0  0.074693
454.287500   52.0  0.089915                 52.0  0.083306
478.062500   53.0  0.097393                 53.0  0.091452
501.837500   54.0  0.104448                 54.0  0.099334
525.612500   55.0  0.111239                 55.0  0.107022
549.387500   56.0  0.117911                 56.0  0.115016
573.162500   57.0  0.124606                 57.0  0.123350
596.937500   58.0  0.133210                 58.0  0.133748
620.712500   59.0  0.141760                 59.0  0.144153
644.487500   60.0  0.149839                 60.0  0.153861
668.262500   61.0  0.157540                 61.0  0.162834
692.037500   62.0  0.164920                 62.0  0.171175
715.812500   63.0  0.172149                 63.0  0.179192
739.587500   64.0  0.179386                 64.0  0.187358
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -24.894165   0.000000 -24.894165        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -24.637442   0.000001   
6.244000e-04 2     -24.894180   0.000034 -24.894146 -24.637459   0.000025   
8.759000e-04 3     -24.894186   0.000047 -24.894138 -24.637465   0.000035   
1.228600e-03 4     -24.894194   0.000067 -24.894127 -24.637474   0.000049   
1.723400e-03 5     -24.894205   0.000094 -24.894111 -24.637487   0.000069   
2.417400e-03 6     -24.894221   0.000133 -24.894088 -24.637504   0.000097   
3.390900e-03 7     -24.894243   0.000187 -24.894055 -24.637529   0.000138   
4.756500e-03 8     -24.894273   0.000264 -24.894009 -24.637563   0.000196   
6.672000e-03 9     -24.894315   0.000370 -24.893945 -24.637611   0.000279   
9.358900e-03 10    -24.894373   0.000516 -24.893857 -24.637677   0.000396   
1.312780e-02 11    -24.894455   0.000715 -24.893740 -24.637771   0.000562   
1.841450e-02 12    -24.894571   0.000981 -24.893590 -24.637904   0.000794   
2.583020e-02 13    -24.894736   0.001331 -24.893405 -24.638094   0.001116   
3.623230e-02 14    -24.894978   0.001783 -24.893196 -24.638369   0.001556   
5.082340e-02 15    -24.895344   0.002352 -24.892992 -24.638775   0.002147   
7.129060e-02 16    -24.895903   0.003098 -24.892806 -24.639376   0.002963   
1.000000e-01 17    -24.896714   0.004224 -24.892490 -24.640227   0.004215   
1.402710e-01 18    -24.897808   0.006021 -24.891787 -24.641360   0.006210   
1.967600e-01 19    -24.899241   0.008732 -24.890509 -24.642817   0.009177   
2.759970e-01 20    -24.901045   0.012661 -24.888383 -24.644615   0.013243   
3.871000e-01 21    -24.903171   0.018269 -24.884902 -24.646691   0.018541   
5.431000e-01 22    -24.905324   0.026365 -24.878959 -24.648755   0.025670   
7.617000e-01 23    -24.906220   0.038425 -24.867795 -24.649511   0.036364   
1.068500e+00 24    -24.902862   0.055775 -24.847087 -24.645900   0.052416   
1.498800e+00 25    -24.894742   0.075877 -24.818865 -24.637478   0.071792   
2.102400e+00 26    -24.884595   0.095855 -24.788739 -24.626825   0.091695   
2.949000e+00 27    -24.873417   0.116654 -24.756763 -24.614859   0.112574   
4.136600e+00 28    -24.861500   0.139125 -24.722374 -24.601720   0.135745   
5.802500e+00 29    -24.849148   0.163983 -24.685165 -24.587576   0.162462   
8.139200e+00 30    -24.836437   0.192165 -24.644272 -24.572229   0.193540   
1.141700e+01 31    -24.823404   0.224267 -24.599137 -24.555635   0.227519   
1.601470e+01 32    -24.811123   0.259918 -24.551205 -24.538932   0.261653   
2.246400e+01 33    -24.802545   0.298896 -24.503649 -24.525663   0.293563   
3.151050e+01 34    -24.800026   0.344531 -24.455494 -24.518785   0.325905   
4.420010e+01 35    -24.804716   0.401849 -24.402867 -24.519617   0.367291   
6.200000e+01 36    -24.822883   0.472684 -24.350199 -24.535527   0.428478   
8.577500e+01 37    -24.867199   0.557171 -24.310028 -24.581523   0.515265   
1.095500e+02 38    -24.928896   0.640956 -24.287940 -24.648804   0.607605   
1.333250e+02 39    -25.004364   0.729158 -24.275206 -24.733168   0.705252   
1.571000e+02 40    -25.099094   0.817768 -24.281326 -24.839281   0.801994   
1.808750e+02 41    -25.222514   0.912843 -24.309672 -24.976080   0.903688   
2.046500e+02 42    -25.372555   1.036414 -24.336141 -25.139654   1.031695   
2.284250e+02 43    -25.538276   1.203240 -24.335036 -25.319903   1.202113   
2.522000e+02 44    -25.715900   1.408210 -24.307689 -25.512457   1.410383   
2.759750e+02 45    -25.904635   1.648760 -24.255875 -25.716194   1.653341   
2.997500e+02 46    -26.104258   1.925579 -24.178679 -25.930625   1.930508   
3.235250e+02 47    -26.313137   2.236859 -24.076278 -26.153924   2.239689   
3.473000e+02 48    -26.529927   2.581217 -23.948711 -26.384363   2.579188   
3.710750e+02 49    -26

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')